### Načítanie knižníc

In [42]:
# nacitanie kniznic pre analyzu dat
import pandas as pd
import numpy as np

# nacitanie kniznic pre rozhodovacie stromy
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

# kniznica na zobrazenie frame-u
from IPython.display import IFrame

### Načítanie datasetu

In [44]:
# dataset sa nachadza na tejto adrese: https://www.kaggle.com/bitsofishan/covid19-patient-symptoms
IFrame('https://www.kaggle.com/bitsofishan/covid19-patient-symptoms', width = 800, height = 450)

# u mna je lokalizovany lokalne v projekte
# nacitam jeho cestuip
csv_file_path = "../db/COVID-19-Survey.csv"


In [45]:
# nazvy stlpcov, s ktorymi budeme pracovat
# col_names = ['age', 'gender','body-temperature', 'dry-cough', 'sour-throat', 'breathing-problem', 'wearness', 'drowsiness', 'pain-in-chest', 'diabetes','change-in-sns','travel-history']

# nacitame dataset
covid_csv = pd.read_csv(csv_file_path, sep=";")

In [46]:
covid_csv.head(5)

,age,gender,body-temperature,dry-cough,sour-throat,breathing-problem,pain-in-chest,weakness,drowsiness,diabetes,change-in-sns,travel-history
0,20,F,37,0,0,0,0,0,0,0,0,0
1,19,F,"37,2",1,0,0,0,0,0,0,0,0
2,55,M,"38,9",1,1,1,1,1,1,0,1,0
3,40,M,"37,8",0,0,0,1,0,1,1,1,1
4,33,F,"37,3",0,1,1,0,0,0,0,1,1


In [47]:
covid_csv.describe()

,age,dry-cough,sour-throat,breathing-problem,pain-in-chest,weakness,drowsiness,diabetes,change-in-sns,travel-history
count,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000
mean,36.456693,0.488189,0.551181,0.433071,0.370079,0.409449,0.425197,0.244094,0.629921,0.464567
std,14.052484,0.501840,0.499343,0.497463,0.484738,0.493680,0.496331,0.431250,0.484738,0.500718
min,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,33.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,45.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000
max,80.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [48]:
# v datasete sa nachadza okrem ciselnych aj jeden stlpec s kategorialnou premennou
# get dummies prehodi kategorialne premenne na ciselne
new_covid_csv = pd.get_dummies(covid_csv, columns=["gender"], drop_first=False)

In [49]:
new_covid_csv






,age,body-temperature,dry-cough,sour-throat,breathing-problem,pain-in-chest,weakness,drowsiness,diabetes,change-in-sns,travel-history,gender_F,gender_M
0,20,37,0,0,0,0,0,0,0,0,0,1,0
1,19,"37,2",1,0,0,0,0,0,0,0,0,1,0
2,55,"38,9",1,1,1,1,1,1,0,1,0,0,1
3,40,"37,8",0,0,0,1,0,1,1,1,1,0,1
4,33,"37,3",0,1,1,0,0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,37,"38,9",1,1,1,0,1,0,0,1,0,1,0
123,35,"37,8",0,0,0,1,0,1,1,1,1,0,1
124,22,"37,3",0,0,1,0,0,0,0,1,0,1,0
125,29,"38,9",0,1,1,1,1,1,0,1,0,0,1


In [ ]:
X = new_covid_csv.iloc[:,0:11]
y=np.ravel(new_covid_csv['result'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
